- In this notebook we tried to extend the base dataset using Tourpedia dataset form Paris attractions.
- We first a cleaned version of Tourpedia dataset from data.mendeley.com to indentify the subCategory related to our base dataset using human inspection.
- Then we filtered the main Tourpedia dataset by these related subCategories to get a smaller dataset that is more relevant to our base dataset.
- Finally, we use the the details API and reviews API to extend the filtered Tourpedia dataset with more information and create a review dataset.

In [ ]:
import pandas as pd

In [166]:
path = "Tourpedia/Paris.csv"

df_tp_mendeley = pd.read_csv(
    path,
    sep=";",
)

df_tp_mendeley.subCategory.unique().tolist()


# df_tp_mendeley_base_related = df_tp_mendeley[df_tp_mendeley["subCategory"].isin(kept_strict)].reset_index(drop=True)
# print(df_tp_mendeley_base_related.shape)
# df_tp_mendeley_base_related.head(2)

['Hotel Bar',
 'Hotel',
 'Wine Bar',
 'Music Venue',
 'Pub',
 'Bar',
 'Cocktail Bar',
 'Sports Bar',
 'Nightclub',
 'Rock Club',
 'Lounge',
 'Church',
 'Lake',
 'Cemetery',
 'Plaza',
 'Park',
 'Event Space',
 'Temple',
 'Mosque',
 'Historic Site',
 'Garden',
 'Bridge',
 'Museum',
 'Zoo',
 'Art Museum',
 'Harbor / Marina',
 'Island',
 'Art Gallery',
 'History Museum',
 'Theater',
 'Comedy Club',
 'Concert Hall',
 'Strip Club',
 'Multiplex',
 'Arts & Entertainment',
 'Pool',
 'Movie Theater',
 'Stadium',
 'Gay Bar',
 'Radio Station',
 'Neighborhood',
 'Performing Arts Venue',
 'Indie Movie Theater',
 'Public Art',
 'Farm',
 'Scenic Lookout',
 'River',
 'Opera House',
 'Dance Studio',
 'Auditorium',
 'Athletics & Sports',
 'General Entertainment',
 'Hostel',
 'Racetrack',
 'Golf Course',
 'Other Great Outdoors',
 'Brewery',
 'Track Stadium',
 'Other Nightlife',
 'Pizza Place',
 'Trail',
 'Science Museum',
 'Tennis Court',
 'Tennis',
 'Sculpture Garden',
 'Bowling Alley',
 'Military Base',

In [168]:
related_subCategory_set = {
    "Church",
    "Temple",
    "Mosque",
    "Historic Site",
    "Museum",
    "Art Museum",
    "History Museum",
    "Science Museum",
    "Art Gallery",
    "Public Art",
    "Monument / Landmark",
    "Cemetery",
    "Garden",
    "Sculpture Garden",
    "Park",
    "Plaza",
    "Bridge",
}

In [169]:
df_tp = pd.read_csv("Tourpedia/paris-attraction.csv")
df_tp.id
print(df_tp.shape)
df_tp.head(2)

(4351, 12)


,address,category,id,lat,lng,location,name,originalId,polarity,subCategory,details,reviews
0,8 rue de Navarin,attraction,83256,48.87961034033,2.3393825451216,Paris,Hôtel Amour,4ac79395f964a52021b820e3,0,Hotel Bar,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...
1,49 Rue Pierre Charron,attraction,83321,48.869145508329,2.3021912516801,Paris,Pershing Hall,4adcda03f964a520c03121e3,0,Hotel,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...


In [170]:
df_tp_base_related =  df_tp[df_tp.subCategory.isin(related_subCategory_set)]
print(df_tp_base_related.shape)
df_tp_base_related.head(2)

(486, 12)


,address,category,id,lat,lng,location,name,originalId,polarity,subCategory,details,reviews
34,Place de la Madeleine,attraction,83456,48.869753812208,2.3242735862732,Paris,Église de la Madeleine,4adcda09f964a520013421e3,0,Church,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...
36,20 Avenue Rachel,attraction,83461,48.887548749415,2.3304319381714,Paris,Cimetière de Montmartre,4adcda09f964a5200a3421e3,0,Cemetery,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...


In [127]:
df_tp_base_related.to_csv("Tourpedia/paris-attraction-base-related.csv", index=False)

In [174]:
import requests
import time

df_tp_base_related_reviews = pd.DataFrame(columns=[
    'monument_id',
    'language',
    'polarity',
    'rating',
    'source',
    'text',
    'time',
    'wordsCount',
    'details_url'
])
df_tp_base_related_with_details = df_tp_base_related.copy()

for index, row in df_tp_base_related.iterrows():
    details_url = row['details']
    reviews_url = row['reviews']
    
    res_details = requests.get(details_url)
    while res_details.status_code != 200:
        time.sleep(1)
        res_details = requests.get(details_url)
    res_reveiews = requests.get(reviews_url)
    while res_reveiews.status_code != 200:
        time.sleep(1)
        res_reveiews = requests.get(reviews_url)
        
    df_tp_base_related_reviews = pd.concat([df_tp_base_related_reviews, pd.DataFrame([{
        'monument_id': row['id'],
        'language': review['language'],
        'polarity': review['polarity'],
        'rating': review['rating'],
        'source': review['source'],
        'text': review['text'],
        'time': review['time'],
        'wordsCount': review['wordsCount'],
        'details_url': review['details']
    } for review in res_reveiews.json()])], ignore_index=True)
    
    res_details_json = res_details.json()
    # if res_details_json['external_urls']['Facebook'] != "" or res_details_json['external_urls']['GooglePlaces'] != "" or res_details_json['external_urls']['Booking'] != "":
    #     print(res_details_json)
    # break
    df_tp_base_related_with_details.at[index, 'description_en'] = res_details_json['description']['en']
    df_tp_base_related_with_details.at[index, 'description.fr'] = res_details_json['description']['fr']
    df_tp_base_related_with_details.at[index, 'Foursquare.url'] = res_details_json['external_urls']['Foursquare']
    df_tp_base_related_with_details.at[index, 'international_phone_number'] = res_details_json['international_phone_number']
    df_tp_base_related_with_details.at[index, 'phone_number'] = res_details_json['phone_number']
    df_tp_base_related_with_details.at[index, 'polarity'] = res_details_json['polarity']
    df_tp_base_related_with_details.at[index, 'services'] = str(res_details_json['services'])
    df_tp_base_related_with_details.at[index, 'Foursquare.usersCount'] = res_details_json['statistics']['Foursquare']['usersCount']
    df_tp_base_related_with_details.at[index, 'Foursquare.checkinsCount'] = res_details_json['statistics']['Foursquare']['checkinsCount']
    df_tp_base_related_with_details.at[index, 'Foursquare.tipCount'] = res_details_json['statistics']['Foursquare']['tipCount']
    df_tp_base_related_with_details.at[index, 'Foursquare.price'] = res_details_json['statistics']['Foursquare']['price']
    df_tp_base_related_with_details.at[index, 'Foursquare.likes'] = res_details_json['statistics']['Foursquare']['likes']
    df_tp_base_related_with_details.at[index, 'website'] = res_details_json['website']
    df_tp_base_related_with_details.at[index, 'numReviews'] = res_details_json.get('numReviews', None)

# Extracted Fields from the Detials of df_tp
- description.en -> description
- external_urls.Foursquare
- international_phone_number
- phone_number
- polarity
- services (is a list)
- statistics.Foursquare.usersCount
- statistics.Foursquare.checkinsCount
- statistics.Foursquare.tipCount
- statistics.Foursquare.price
- statistics.Foursquare.likes
- website
- numReviews

In [179]:
print(df_tp_base_related_with_details.shape)
df_tp_base_related_with_details.head(2)

(486, 25)


,address,category,id,lat,lng,location,name,originalId,polarity,subCategory,...,international_phone_number,phone_number,services,Foursquare.usersCount,Foursquare.checkinsCount,Foursquare.tipCount,Foursquare.price,Foursquare.likes,website,numReviews
34,Place de la Madeleine,attraction,83456,48.869753812208,2.3242735862732,Paris,Église de la Madeleine,4adcda09f964a520013421e3,4,Church,...,,,[],1956.0,2839.0,9.0,,28.0,http://www.eglise-lamadeleine.com,9.0
36,20 Avenue Rachel,attraction,83461,48.887548749415,2.3304319381714,Paris,Cimetière de Montmartre,4adcda09f964a5200a3421e3,6,Cemetery,...,,,[],636.0,1164.0,6.0,,8.0,,6.0


In [180]:
print(df_tp_base_related_reviews.shape)
df_tp_base_related_reviews.head(2)

(2613, 9)


,monument_id,language,polarity,rating,source,text,time,wordsCount,details_url
0,83456,fr,5,0,Foursquare,Une des trois plus laides églises de Paris ! M...,2011-08-31,21,http://tour-pedia.org/api/getReviewDetails?id=...
1,83456,fr,0,0,Foursquare,Je ne suis pas spécialiste des églises! Mais j...,2011-09-30,37,http://tour-pedia.org/api/getReviewDetails?id=...


In [185]:
df_base = pd.read_csv("paris_monuments_translated.csv")
print(df_base.shape)
df_base.head(2)

(88, 10)


,name,url,short_description,ticket_price,ticket_price_conditions,opening_hours,payment_methods,address,visiting_services,ticket_price_raw
0,Basilique Cathédrale de Saint-Denis,https://www.saint-denis-basilique.fr/,Discover the necropolis of the kings and queen...,17.0,Free for under 26s,From October to March\nMonday to Saturday: 10:...,coins | bank cards | cheques | Culture cheques...,"1, rue de la Légion d'Honneur, 93200 Saint-Denis",Cabin baggage ('cabines') allowed: maximum 40x...,Price: 17 €
1,Oppidum et musée archéologique d'Ensérune,https://www.enserune.fr/,Discover a major Gallo-Roman city located betw...,9.0,Free for those under 26 years old.,Please note: last access to the monument one h...,cash | bank cards | cheques | Culture cheques ...,"2901 route d’Ensérune, 34440 Nissan-lez-Ensérune",Drinking fountain | Toilets | Wheelchair | Pos...,Price: 9 €


# Merging the base dataset with the extended Tourpedia dataset
- Now we want to merge the base dataset with the extracted Tourpedia dataset.
- First we should identify the common rows between the two datasets based on the name, address.

In [294]:
import pandas as pd
from thefuzz import fuzz

df1 = df_base.copy()
df2 = df_tp_base_related_with_details.rename(columns={'website': 'url'}).copy()


df1['fingerprint'] = (df1['name'].fillna('') + " " + df1['address'].fillna('') + " " + df1['url'].fillna('')).str.lower()
df2['fingerprint'] = (df2['name'].fillna('') + " " + df2['address'].fillna('') + " " + df2['url'].fillna('')).str.lower()


df1_fingerprints = df1['fingerprint']
df2_fingerprints = df2['fingerprint']

for idx2, fingerprint2 in df2_fingerprints.items():
    max_similarity = -1
    sim_idx1 = None
    for idx1, fingerprint1 in df1_fingerprints.items():
        similarity = fuzz.token_set_ratio(fingerprint2, fingerprint1)
        if similarity > 80:
            print("similarity:", similarity)
            print("fingerprint1:", fingerprint1)
            print("fingerprint2:", fingerprint2)
        if similarity > max_similarity:
            max_similarity = similarity
            sim_idx1 = idx1
    df2.at[idx2, 'related_index'] = sim_idx1
    df2.at[idx2, 'similarity'] = max_similarity

df1 = df1.drop(columns=['fingerprint'])
df2 = df2.drop(columns=['fingerprint'])
df2['related_index'] = df2['related_index'].astype(int)

similarity: 83
fingerprint1: château de vincennes tour des salves, cour des maréchaux, 75012 paris https://www.chateau-de-vincennes.fr/
fingerprint2: bois de vincennes bois de vincennes 
similarity: 100
fingerprint1: colonne de juillet place de la bastille, 75011 paris https://www.colonne-de-juillet.fr/
fingerprint2: place de la bastille place de la bastille 
similarity: 83
fingerprint1: abbaye de cluny palais du pape gélase, place du 11 août 1944, 71250 cluny https://www.cluny-abbaye.fr/
fingerprint2: place du palais royal place du palais royal 
similarity: 83
fingerprint1: domaine national du palais-royal 8 rue montpensier, 75001 paris https://www.domaine-palais-royal.fr/
fingerprint2: place du palais royal place du palais royal 
similarity: 85
fingerprint1: arc de triomphe place charles de gaulle, 75008 paris https://www.paris-arc-de-triomphe.fr/
fingerprint2: place de l’étoile place charles de gaulle 
similarity: 100
fingerprint1: village fortifié de mont-dauphin pavillon de l'horl

In [288]:
index = 1
threshold = 85
df2[df2['similarity'] > threshold].iloc[[index], :]

,address,category,id,lat,lng,location,name,originalId,polarity,subCategory,...,services,Foursquare.usersCount,Foursquare.checkinsCount,Foursquare.tipCount,Foursquare.price,Foursquare.likes,url,numReviews,related_index,similarity
382,Place Vauban,attraction,85294,48.853505642662,2.3123645782471,Paris,Place Vauban,4ba3fec3f964a5209b7538e3,8,Plaza,...,[],400.0,991.0,3.0,,3.0,,3.0,9,100.0


In [292]:
df1.iloc[[df2[df2['similarity'] > threshold].related_index.tolist()[index]], :]
# df1.iloc[[9], :]

,name,url,short_description,ticket_price,ticket_price_conditions,opening_hours,payment_methods,address,visiting_services,ticket_price_raw
9,Village fortifié de Mont-Dauphin,https://www.village-fortifie-montdauphin.fr/,Discover a fortified village inscribed on the ...,7.0,Reserve your guided tour of the fortifications,The fortifications and military buildings of t...,cash | bank cards | cheques | holiday cheques ...,"Pavillon de l'Horloge, Ticket office/shop, 1 P...",Parking for cars and buses is mandatory (free)...,Price: 7 €


In [295]:
import pandas as pd
from thefuzz import fuzz

# 1. Combine all your DataFrames into one
df = pd.concat([df2], ignore_index=True)

# 2. Create a unique "fingerprint" string for each row to compare
#    combining key fields like Name and Address improves accuracy
df['fingerprint'] = (df['name'].fillna('') + " " + df['address'].fillna('')).str.lower()

# 3. Find Fuzzy Duplicates
#    We iterate and check if the current row matches any *previous* row with >85% similarity
keep_indices = []
seen_fingerprints = []

for idx, row in df.iterrows():
    # check similarity against already approved rows
    # 'token_set_ratio' handles out-of-order words well (e.g., "Inc Apple" vs "Apple Inc")
    is_duplicate = False
    for seen in seen_fingerprints:
        if score := fuzz.token_set_ratio(row['fingerprint'], seen) > 85:
            is_duplicate = True
            print("similarity score:", score)
            print("fingerprint1:", seen)
            print("fingerprint2:", row['fingerprint'])
    
    if not is_duplicate:
        keep_indices.append(idx)
        seen_fingerprints.append(row['fingerprint'])

# 4. Result: A clean DataFrame and a separate DataFrame of the removed duplicates
clean_df = df.loc[keep_indices].drop(columns=['fingerprint'])
duplicates_df = df.drop(keep_indices).drop(columns=['fingerprint'])

similarity score: True
fingerprint1: palais royal place du palais royal
fingerprint2: place du palais royal place du palais royal
similarity score: True
fingerprint1: église de la madeleine place de la madeleine
fingerprint2: place de la madeleine place de la madeleine
similarity score: True
fingerprint1: palais royal place du palais royal
fingerprint2: jardin du palais royal 2 place colette
similarity score: True
fingerprint1: place des ternes place des ternes
fingerprint2: église saint-ferdinand-des-ternes place tristan bernard
similarity score: True
fingerprint1: place saint-ferdinand place saint-ferdinand
fingerprint2: église saint-ferdinand-des-ternes place tristan bernard
similarity score: True
fingerprint1: place de l'odéon place de l'odéon
fingerprint2: carrefour de l'odéon place henri mondor
similarity score: True
fingerprint1: place des victoires place des victoires
fingerprint2: basilique notre-dame-des-victoires place des petits pères
similarity score: True
fingerprint1: pl